In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)


c:\Users\Admin\Data\multimodal-rag-baseline


In [2]:
from configs.config import *
from libs.common import *
from utils.mongodb_connection import *
from utils.rag_utils import *
from utils.format_utils import *
load_dotenv(find_dotenv())

c:/Users/Admin/Data/multimodal-rag-baseline


c:\Users\Admin\miniconda3\envs\kyanon_digital\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
mongo_uri = os.getenv('MONGO_URI')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
collection = "kyanon_digital"

mongo_client = get_mongo_client(mongo_uri)
db = mongo_client['Kyanon']
collection = db['RAG']

Connection to MongoDB successful


In [4]:
# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")
embedding_model = embedding_model.to('cuda')

print("Model is using GPU:", next(embedding_model.parameters()).is_cuda)

Model is using GPU: True


In [5]:
import camelot

pdf_path = "content.pdf"

tables = camelot.read_pdf(pdf_path, pages="all", flavor='lattice')
print(f"Tổng cộng: {tables.n} bảng được tìm thấy.")

table_strings = []

for i, table in enumerate(tables):
    table_str = table.df.to_markdown(index=False)  # Dễ đọc hơn với index=False
    table_strings.append(table_str)
    print(f"\n--- Bảng {i + 1} ---")
    print(table_str)



c:\Users\Admin\miniconda3\envs\kyanon_digital\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


Tổng cộng: 10 bảng được tìm thấy.

--- Bảng 1 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | Elected to the Hall of Fame on this ballot (named |
|     | in bold italics).                                 |
|     | Elected subsequently, as of 2025 (named in plain  |
|     | italics).                                         |
|     | Renominated for the 2019 BBWAA election by        |
|     | adequate performance on this ballot and has not   |
|     | subsequently been eliminated.                     |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expiration on subsequent      |

--- Bảng 2 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | ballots.                                          |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expira

In [6]:
import requests
import json

def generate_response(prompt):
    """
    Gửi prompt đến Ollama và nhận về một đoạn phản hồi hoàn chỉnh.
    
    Tham số:
        prompt (str): Câu hỏi hoặc yêu cầu bạn muốn gửi đến mô hình.

    Trả về:
        str: Phản hồi hoàn chỉnh từ mô hình.
    """
    full_response = ""
    context = f"""
    You are given a single table fragment extracted from a PDF document using OCR. This fragment may be part of a larger table that was split across multiple pages due to formatting or page layout.
    Your task is to generate a detailed and comprehensive summary of the content in this table fragment.
    The summary should clearly describe:
        The main subject or topic of the table
        Key columns and their meanings
        Important patterns, trends, or observations in the data
        Any notable values or anomalies
        Contextual information needed to understand the data
    Write the summary in 1–3 full sentences , using clear and precise language.
    If applicable, mention that this fragment appears to be part of a larger table, and include any inferred continuity from the data.
    {prompt}
    """
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gemma3:latest",
        "prompt": context,
        "stream": True
    }

    with requests.post(url, json=payload, stream=True) as response:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line)
                    if 'response' in data:
                        chunk = data['response']
                        full_response += chunk
                    if data.get('done', False):
                        break
                except json.JSONDecodeError:
                    continue  

    return full_response.strip()

In [7]:
for i in tables:
    print(i.df)
    table_summary = generate_response(i.df.to_markdown(index=False))
    print(table_summary)
    embedding = get_embedding(embedding_model,table_summary)
    
    chunk_tables = []
    
    document = {
        "embedding": embedding,
        "chunk": table_summary,
        "table": i.df.to_markdown(index=False)
        
            
    }
    collection.insert_one(document)

  0                                                  1
0    Elected to the Hall of Fame on this ballot (na...
1    Elected subsequently, as of 2025 (named in pla...
2    Renominated for the 2019 BBWAA election by\nad...
3    Eliminated from annual BBWAA consideration by\...
This table fragment appears to be documenting a baseball writer's association (BBWAA) Hall of Fame ballot selection history, tracking a player's status as either "Elected" (bold italics) or "Eliminated" (plain italics) over time. The table records the year of election, along with details about the player’s performance on the ballot and whether they remained eligible for future consideration, highlighting a pattern of initial election followed by potential elimination based on performance or ballot expiration. Given the focus on election years, it’s likely this fragment represents a portion of a larger table detailing a player’s entire Hall of Fame voting record.
   0                                                  

In [8]:
all_tables_combined = ""

for i, table in enumerate(tables):
    table_str = table.df.to_markdown(index=False)  
    all_tables_combined += f"\n\n--- Bảng {i + 1} ---\n"
    all_tables_combined += table_str



print("\n--- Toàn bộ bảng đã gộp ---")
print(all_tables_combined)


--- Toàn bộ bảng đã gộp ---


--- Bảng 1 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | Elected to the Hall of Fame on this ballot (named |
|     | in bold italics).                                 |
|     | Elected subsequently, as of 2025 (named in plain  |
|     | italics).                                         |
|     | Renominated for the 2019 BBWAA election by        |
|     | adequate performance on this ballot and has not   |
|     | subsequently been eliminated.                     |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expiration on subsequent      |

--- Bảng 2 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | ballots.                                          |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expiration

In [9]:
query = "What is The BBWAA election rules?"
source_information = get_search_result(embedding_model, query.lower(), collection)
source_information

{'chunk': 'This table fragment details the results of ballots considered by the Baseball Writers Association of America (BBWAA) for annual awards. Key columns include performance metrics and ballot status. Notably, the table reveals instances where candidates were eliminated due to either poor performance or ballot expiration, with the latter potentially indicating a lack of statistical significance in their performance. Given the presence of both first-time candidates and those eliminated in previous years, this fragment appears to be part of a larger table providing comprehensive information about the BBWAA ballot process.', 'table': '| 0   | 1                                                 |\n|:----|:--------------------------------------------------|\n|     | ballots.                                          |\n|     | Eliminated from annual BBWAA consideration by     |\n|     | poor performance or expiration on this ballot.    |\n| †   | First time on the BBWAA ballot.           

"\n 1) Summary: This table fragment details the results of ballots considered by the Baseball Writers Association of America (BBWAA) for annual awards. Key columns include performance metrics and ballot status. Notably, the table reveals instances where candidates were eliminated due to either poor performance or ballot expiration, with the latter potentially indicating a lack of statistical significance in their performance. Given the presence of both first-time candidates and those eliminated in previous years, this fragment appears to be part of a larger table providing comprehensive information about the BBWAA ballot process.\n Table: | 0   | 1                                                 |\n|:----|:--------------------------------------------------|\n|     | ballots.                                          |\n|     | Eliminated from annual BBWAA consideration by     |\n|     | poor performance or expiration on this ballot.    |\n| †   | First time on the BBWAA ballot.         

In [10]:
combined_information = f"Become an expert on the concepts on Wikipedia. Customer's question: {query}\nAnswer the question based on the following document information: {source_information}."
print('==== LLMs Inference')
print(combined_information)

==== LLMs Inference
Become an expert on the concepts on Wikipedia. Customer's question: What is The BBWAA election rules?
Answer the question based on the following document information: 
 1) Summary: This table fragment details the results of ballots considered by the Baseball Writers Association of America (BBWAA) for annual awards. Key columns include performance metrics and ballot status. Notably, the table reveals instances where candidates were eliminated due to either poor performance or ballot expiration, with the latter potentially indicating a lack of statistical significance in their performance. Given the presence of both first-time candidates and those eliminated in previous years, this fragment appears to be part of a larger table providing comprehensive information about the BBWAA ballot process.
 Table: | 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | ballots.                                 

In [11]:
from huggingface_hub import login

api_token = os.getenv('HUGGINGE_FACE_TOKEN') 
login(api_token)



In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.89s/it]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [13]:
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=100)
print(tokenizer.decode(response[0]))

<bos>Become an expert on the concepts on Wikipedia. Customer's question: What is The BBWAA election rules?
Answer the question based on the following document information: 
 1) Summary: This table fragment details the results of ballots considered by the Baseball Writers Association of America (BBWAA) for annual awards. Key columns include performance metrics and ballot status. Notably, the table reveals instances where candidates were eliminated due to either poor performance or ballot expiration, with the latter potentially indicating a lack of statistical significance in their performance. Given the presence of both first-time candidates and those eliminated in previous years, this fragment appears to be part of a larger table providing comprehensive information about the BBWAA ballot process.
 Table: | 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | ballots.                                          |
|   

In [15]:
import json

generated_qa_pairs = json.load(open(f"{data_dir}/generated_qa_pairs.json"))

In [21]:
def calculate_recall(qa, k = 5):
    question = qa["question"]
    # answer = qa["answer"]
    # context = qa["context"]
    
    retrieved_top_k_items = get_search_result(embedding_model, question, collection)

    ground_truth_source_ids = [f"{qa['source']}_table_{qa['table_idx']}"]
    relevant = sum(1 for item in retrieved_top_k_items if item.metadata.get("source_table_idx") in ground_truth_source_ids)
    return relevant/len(ground_truth_source_ids) if len(ground_truth_source_ids) > 0 else 0

def calculate_precision(qa, k=5):
    question = qa["question"]
    # answer = qa["answer"] 
    # context = qa["context"]
    
    retrieved_top_k_items = get_search_result(embedding_model, question, k)
    ground_truth_source_ids = [f"{qa['source']}_table_{qa['table_idx']}"]
    num_retrieved_and_relevant = 0
    if retrieved_top_k_items: # Đảm bảo có item được truy xuất để kiểm tra
        num_retrieved_and_relevant = sum(
            1 
            for item in retrieved_top_k_items
            if item.metadata.get("source_table_idx") in ground_truth_source_ids
        )

    if k > 0:
        precision_at_k = num_retrieved_and_relevant / k
    else:
        precision_at_k = 0

    return precision_at_k
    

In [22]:
recall_k = []

for qa in generated_qa_pairs:
    recall_k.append(calculate_recall(qa, k = 5))
    
print(sum(recall_k) / len(recall_k))
    

{'chunk': "This table fragment details the status of baseball players on the Baseball Writers' Association of America (BBWAA) All-Star Game ballot, indicating whether they were first-time voters or had been previously eliminated due to poor performance or ballot expiration.  The columns likely represent the player’s name and their ballot status, with “First time on the BBWAA ballot” signifying a new entry and “Eliminated…poor performance” denoting removal from consideration based on performance. Given the presence of first-time ballots, this fragment likely represents a portion of a larger table tracking player voting trends across multiple years for the All-Star Game selection.", 'table': '| 0   | 1                                                 |\n|:----|:--------------------------------------------------|\n|     | ballots.                                          |\n|     | Eliminated from annual BBWAA consideration by     |\n|     | poor performance or expiration on this ballot.  

AttributeError: 'str' object has no attribute 'metadata'

In [ ]:
recall_k_summary = []

for qa in generated_qa_pairs:
    recall_k_summary.append(calculate_recall(qa, summary_vectorstore))
    
print(sum(recall_k_summary) / len(recall_k_summary))
    

In [ ]:
precision_k_detail = []
for qa in generated_qa_pairs:
    precision_k_detail.append(calculate_precision(qa, detail_vectorstore))
    
print(sum(precision_k_detail) / len(precision_k_detail))

In [ ]:
precision_k_summary = []
for qa in generated_qa_pairs:
    precision_k_summary.append(calculate_precision(qa, summary_vectorstore))
    
print(sum(precision_k_summary) / len(precision_k_summary))

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_metric

# Load các chỉ số đánh giá
bleu_metric = load_metric("bleu")
rouge_metric = load_metric("rouge")

def exact_match_score(prediction, ground_truth):
    return int(prediction.strip() == ground_truth.strip())

def f1_score(prediction, ground_truth):
    from collections import Counter
    pred_tokens = prediction.lower().split()
    gt_tokens = ground_truth.lower().split()
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    f1 = 2 * precision * recall / (precision + recall)
    return f1

def evaluate_rag_model(jsonl_path, model_name_or_path):
    # Khởi tạo tokenizer và model
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path).to("cuda")

    em_scores = []
    f1_scores = []
    bleu_scores = []
    rouge_1_scores = []
    rouge_2_scores = []
    rouge_l_scores = []

    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)

            question = data["question"]
            context = data["context"]
            answer = data["answer"]

            # Kết hợp câu hỏi và ngữ cảnh làm đầu vào mô hình
            combined_input = f"Question: {question}\nContext: {context}"

            # Tokenize và generate
            input_ids = tokenizer(combined_input, return_tensors="pt").to("cuda")
            output_ids = model.generate(**input_ids, max_new_tokens=100)
            predicted_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            print(f"Question: {question}")
            print(f"Predicted: {predicted_answer}")
            print(f"Ground Truth: {answer}")
            print("-" * 50)

            # Tính toán các chỉ số
            em = exact_match_score(predicted_answer, answer)
            f1 = f1_score(predicted_answer, answer)

            # BLEU (cần định dạng dưới dạng list of tokens)
            pred_tokens = predicted_answer.split()
            ref_tokens = [answer.split()]
            bleu = bleu_metric.compute(predictions=[pred_tokens], references=[ref_tokens])['bleu']

            # ROUGE
            rouge = rouge_metric.compute(predictions=[predicted_answer], references=[answer])
            rouge_1 = rouge['rouge1'].mid.fmeasure
            rouge_2 = rouge['rouge2'].mid.fmeasure
            rouge_l = rouge['rougeL'].mid.fmeasure

            # Lưu lại
            em_scores.append(em)
            f1_scores.append(f1)
            bleu_scores.append(bleu)
            rouge_1_scores.append(rouge_1)
            rouge_2_scores.append(rouge_2)
            rouge_l_scores.append(rouge_l)

    # Trung bình tất cả các chỉ số
    results = {
        "EM": sum(em_scores) / len(em_scores),
        "F1": sum(f1_scores) / len(f1_scores),
        "BLEU": sum(bleu_scores) / len(bleu_scores),
        "ROUGE-1": sum(rouge_1_scores) / len(rouge_1_scores),
        "ROUGE-2": sum(rouge_2_scores) / len(rouge_2_scores),
        "ROUGE-L": sum(rouge_l_scores) / len(rouge_l_scores)
    }

    return results

In [ ]:
results = evaluate_rag_model("path/to/your/data.jsonl", "facebook/rag-token-nq")
print(results)